Блокнот к статье [Сколько времени валюты проводят в трендах](https://empirix.ru/skolko-valyuty-provodyat-v-trendah/)

# Trend vs. Flat

In [ ]:
# !pip install matplotlib
# !pip install pandas
# !pip install yfinance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

## Ticker from finance.yahoo.com

In [ ]:
ticker = "GBPUSD=X"
date_from = '2003-01-01'
date_to = pd.Timestamp.today().date()
# # Or select custom date
# date_to = '2022-09-23'

df = yf.download(ticker, start=date_from, end=date_to, progress=False)
df

In [ ]:
# Update DF by removing redundant columns
df = df['Close'].to_frame()
# Remove time portion from index
df.index = df.index.date

df

In [ ]:
# Simple moving average - 3
sma_1 = 20

mult = 1.75

sma_2 = int(sma_1 * mult)
sma_3 = int(sma_2 * mult)

print(sma_1, sma_2, sma_3)

## Dataframe, 3 SMAs

In [ ]:
# Calculate SMA using .rolling(window).mean()
for sma in [sma_1, sma_2, sma_3]:
    df[f'SMA{sma}'] = df['Close'].rolling(sma).mean()

# Remove null values
df.dropna(inplace=True)

df.head()

## Plot chart

In [ ]:
df[['Close',
    f'SMA{sma_1}', 
    f'SMA{sma_2}', 
    f'SMA{sma_3}']].plot(label=ticker, figsize=(16, 9))

# Set the title and axis label
plt.title(ticker, fontsize=16)
plt.xlabel('Year-Month', fontsize=15)
plt.ylabel('Price', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.show()

### Plot a subset for more visibility

In [ ]:
# Make a smaller subset
sub_date_from = pd.to_datetime('2019-01-01')
sub_date_to = pd.to_datetime('2019-12-31')

df_sub = df.loc[(pd.to_datetime(df.index) >= sub_date_from)
                & (pd.to_datetime(df.index) <= sub_date_to)]
df_sub.tail()

In [ ]:
df_sub[['Close',
    f'SMA{sma_1}', 
    f'SMA{sma_2}', 
    f'SMA{sma_3}']].plot(label=ticker, figsize=(16, 9))

# Set the title and axis label
plt.title(ticker, fontsize=16)
plt.xlabel('Year-Month', fontsize=15)
plt.ylabel('Price', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.show()

## Calculations: trend vs. flat

In [ ]:
# Calculate states: trend (up, down) of flat
df['uptrend'] = (
    ((df[f'SMA{sma_1}'] > df[f'SMA{sma_2}']) 
     & (df[f'SMA{sma_2}'] > df[f'SMA{sma_3}']))).astype(int)

df['downtrend'] = (
    ((df[f'SMA{sma_1}'] < df[f'SMA{sma_2}'])          
     & (df[f'SMA{sma_2}'] < df[f'SMA{sma_3}']))).astype(int)

df['flat'] = ((df['uptrend'] == 0) & (df['downtrend'] == 0)).astype(int)
df[470:477]

In [ ]:
# % of time in each state
uptr = df['uptrend'].sum() / len(df)
dntr = df['downtrend'].sum() / len(df)
flat = df['flat'].sum() / len(df)

print(f"Uptrend: {uptr}\nDowntrend: {dntr}\nFlat: {flat}")

## Trend vs. Flat pie chart

In [ ]:
values = [uptr + dntr, flat]
labels = ["Trend", "Flat"]
myexplode = [0, 0.2]
plt.pie(values, labels=labels, startangle=90, explode=myexplode, 
        autopct="%1.1f%%")
plt.title(f"{ticker}, Trend, Flat")
plt.show()

<hr>

# Scan Tickers

In [ ]:
def trend_flat(df_main, date_from, date_to, sma_1, sma_mult):
    df = df_main.copy(deep=True)
    """
    Take df, add smas, compute time in trend/flat.
    Return stats: smas list, time in trend, time in flat,
    date from, date to.
    """
    
    # Calculate SMA using .rolling(window).mean()
    sma_2 = int(sma_1 * sma_mult)
    sma_3 = int(sma_2 * sma_mult)
    for sma in [sma_1, sma_2, sma_3]:
        df[f'SMA{sma}'] = df['Close'].rolling(sma).mean()

    # Remove null values
    df.dropna(inplace=True)
    
    # Tag days as 1 or 0, spent in trend or flat
    df['uptrend'] = (
    ((df[f'SMA{sma_1}'] > df[f'SMA{sma_2}']) 
     & (df[f'SMA{sma_2}'] > df[f'SMA{sma_3}']))).astype(int)

    df['downtrend'] = (
        ((df[f'SMA{sma_1}'] < df[f'SMA{sma_2}'])          
         & (df[f'SMA{sma_2}'] < df[f'SMA{sma_3}']))).astype(int)

    df['flat'] = ((df['uptrend'] == 0) 
                  & (df['downtrend'] == 0)).astype(int)

    # % in each state
    uptr = df['uptrend'].sum() / len(df)
    dntr = df['downtrend'].sum() / len(df)
    flat = df['flat'].sum() / len(df)
    
    # Dates
    dt_from = df.index[0]
    dt_to = df.index[len(df)-1]
    
    return (f"{sma_1}_{sma_2}_{sma_3}",
            uptr + dntr, flat, dt_from, dt_to)

In [ ]:
tickers = ["EURUSD=X", "GBPUSD=X", "AUDUSD=X", 
           "NZDUSD=X", "CAD=X", "JPY=X", 
           "EURJPY=X", "GBPJPY=X", "EURGBP=X"]

date_from = '2000-01-01'
date_to = '2022-09-23'
sma_mult = 1.75

smas = [5, 10, 20, 30, 50, 100, 150, 200]

In [ ]:
for ticker in tickers:
    # Download daily prices
    df = yf.download(ticker, start=date_from, end=date_to,
                     progress=False)

    # Update DF by removing redundant columns
    df = df['Close'].to_frame()

    # Remove time portion from index
    df.index = df.index.date
    
    result = {"SMAs": [], "trend": [], "flat": []}
    
    for i in smas:
        tr_fl = trend_flat(df, date_from, date_to, i, sma_mult)
        result["SMAs"].append(tr_fl[0])
        result["trend"].append(tr_fl[1])
        result["flat"].append(tr_fl[2])
        
    print(f'\n\nTICKER: {ticker}')
    print(f'DATES: {tr_fl[3]} - {tr_fl[4]}')
    time_in_trend = round(sum(result["trend"]) 
                          / len(result["trend"]) * 100, 2)
    print(f'TREND, AVERAGE: {time_in_trend}%')

    r_df = pd.DataFrame(result)
    r_df.plot.bar(x='SMAs', stacked=True)
    plt.title(f"{ticker}, Trend vs. Flat")
    plt.show()